# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import wandb

In [2]:
!wandb login xyz

Successfully logged in to Weights & Biases!


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Adam/.netrc


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [3]:
env = UnityEnvironment(file_name="./Tennis_Windows_x86_64/Tennis.exe")
# env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [6]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        
        # print (actions)
        
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.10000000149011612
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.10000000149011612
Score (max over agents) from episode 5: 0.0


### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# import torch & wandb
import torch
import wandb

# import agent
from ddpg_tennis_agent import Agent

# import discipline (training algorithm)
from discipline import train_ddpg_tennis, play_ddpg_tennis, load_agent, save_agent

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

import logging
logger = logging.getLogger('wandb')
logger.setLevel(logging.ERROR)

Hyperparameters

In [17]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 128        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 5e-2              # for soft update of target parameters
LR_ACTOR = 1e-3         # learning rate of the actor 
LR_CRITIC = 1e-3        # learning rate of the critic
WEIGHT_DECAY = 0   # L2 weight decay
OU_SIGMA = 0.19
OU_THETA = 0.13
LEARN_INTERVAL = 1
LEARN_NUM = 1

config = {
    "BUFFER_SIZE": BUFFER_SIZE,
    "BATCH_SIZE": BATCH_SIZE,
    "GAMMA": GAMMA,
    "TAU": TAU,
    "LR_ACTOR": LR_ACTOR,
    "LR_CRITIC": LR_CRITIC,
    "WEIGHT_DECAY": WEIGHT_DECAY,
    "OU_SIGMA": OU_SIGMA,
    "OU_THETA" : OU_THETA,
    "LEARN_INTERVAL": LEARN_INTERVAL,
    "LEARN_NUM": LEARN_NUM
}

In [18]:
# new run
wandb.init(config=config, project="tennis_ddpg_multi_agent")

W&B Run: https://app.wandb.ai/adam_blvck/tennis_ddpg_multi_agent/runs/mf2rkms1

Create 2 new agents, each with their own neural networks and memory containing experience

In [19]:
# agent 0
agent_0 = Agent( device, state_size, num_agents, action_size, 0, \
                BUFFER_SIZE, BATCH_SIZE, GAMMA, TAU, LR_ACTOR, LR_CRITIC, WEIGHT_DECAY, \
                 LEARN_INTERVAL, LEARN_NUM, OU_SIGMA, OU_THETA)
# agent 1
agent_1 = Agent( device, state_size, num_agents, action_size, 0, \
                BUFFER_SIZE, BATCH_SIZE, GAMMA, TAU, LR_ACTOR, LR_CRITIC, WEIGHT_DECAY, \
                 LEARN_INTERVAL, LEARN_NUM, OU_SIGMA, OU_THETA)

# watch agent
wandb.watch(agent_0.actor_local)
wandb.watch(agent_0.critic_local)
wandb.watch(agent_1.actor_local)
wandb.watch(agent_1.critic_local)

In [20]:
# train
scores = train_ddpg_tennis(env, [agent_0, agent_1], num_agents, brain_name, \
                           checkpoint_name='checkpoint_tennis')

Episode 50	Average Score: 0.00	Max score: 0.00
Episode 100	Average Score: 0.01	Max score: 0.00
Episode 150	Average Score: 0.03	Max score: 0.10
Episode 200	Average Score: 0.03	Max score: 0.09
Episode 250	Average Score: 0.04	Max score: 0.00
Episode 300	Average Score: 0.05	Max score: 0.20
Episode 350	Average Score: 0.05	Max score: 0.00
Episode 400	Average Score: 0.06	Max score: 0.00
Episode 450	Average Score: 0.07	Max score: 0.09
Episode 500	Average Score: 0.07	Max score: 0.00
Episode 550	Average Score: 0.08	Max score: 0.10
Episode 600	Average Score: 0.10	Max score: 0.10
Episode 650	Average Score: 0.11	Max score: 0.00
Episode 700	Average Score: 0.12	Max score: 0.10
Episode 750	Average Score: 0.19	Max score: 0.30
Episode 800	Average Score: 0.24	Max score: 0.30
Episode 850	Average Score: 0.27	Max score: 0.50
Episode 900	Average Score: 0.28	Max score: 0.00
Episode 950	Average Score: 0.32	Max score: 0.00
Episode 1000	Average Score: 0.33	Max score: 0.50
Episode 1050	Average Score: 0.36	Max sco

NameError: name 'agent' is not defined

In [28]:
save_agent([agent_0, agent_1], "checkpoint_tennis")

## See the agents playing against themselves

In [30]:
load_agent([agent_0, agent_1], "checkpoint_tennis")

In [31]:
play_ddpg_tennis(env, agent_0, agent_1, num_agents, brain_name)

deque([1.1000000163912773], maxlen=100)
